### RNN, GRU, LSTM을 이용하여 감성분석 해보기

In [2]:
#한국어 라이브러리 konlpy 다운로드
! pip install konlpy

In [3]:


!sudo apt-get install -y fonts-nanum #나눔 글꼴 폰트 패키지를 설치하고,
!sudo fc-cache -fv # 시스템 폰트 캐시를 재설정하며
!rm ~/.cache/matplotlib -rf # matplotlib의 캐시 디렉토리를 삭제하여 새로고침하는 작업을 수행

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 45 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 0s (40.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 121752 files and direc

In [4]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

!apt -qq -y install fonts-nanum

import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic')
#mpl.font_manager._rebuild()

# https://pinkwink.kr/1255 / https://teddylee777.github.io/colab/colab-korean 한글 문제 대응 참고

fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
#필요한 라이브러리 다운로드
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
# 오류(SettingWithCopyError 발생)
pd.set_option('mode.chained_assignment', 'raise') # SettingWithCopyError

# 경고(SettingWithCopyWarning 발생, 기본 값입니다)
pd.set_option('mode.chained_assignment', 'warn') # SettingWithCopyWarning

# 무시
pd.set_option('mode.chained_assignment',  None) # <==== 경고를 끈다

In [16]:
df = pd.read_table('/content/ratings.txt')
df

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


#### 중복 데이터가 있다면 제거

In [17]:
#중복값 제거
df.drop_duplicates('document',keep='first')
df.shape

(200000, 3)

중복 데이터가 없다.

#### Document가 존재하지 않는 것 제거

In [25]:
# Document 중에 값이 없는 것이 존재
db= df.isnull().any()
db

id          False
document     True
label       False
dtype: bool

In [26]:
# NaN값이 존재하므로 제거 해줌
df.dropna()


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


#### 훈련, 테스트셋 분리

In [29]:
# training과 validation 분배
train, val = train_test_split(df, test_size = 0.25, random_state = 101)
train.shape, val.shape

((150000, 3), (50000, 3))

In [30]:
train['label'].value_counts()

label
0    75086
1    74914
Name: count, dtype: int64

#### 데이터 정제 - 훈련, 테스트셋에 모두 적용 -> 정규표현식 사용하여 한글 제외 모두 제거

In [34]:
#한글 제외 모두 제거
#Train set
train['document'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train.head()

,id,document,label
172646,10017356,이런거 안남기는데 솔직히 억지가 너무 심했다. 여자친구랑 다투고 할거없어서 봤는데도...,0
143089,7874090,내용이 너무 지루하다....... 재미없다 굿닥터로 채널 돌림,0
170420,5289473,뭣같은 다큐멘터리도 구천원씩 주고 극장에서 보게 해야겠냐?? 걍 티비에서 방영 하라,0
188793,2259841,이게 영화냐?,0
56748,6698652,인간의 본성에 대한 철처한 해부.,1


In [48]:
#한글 제외 모두 제거
#Validation set
val['document'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# val['document'].replace('', np.nan, inplace=True)
val.head()

,id,document,label
13588,9818708,오랜만에 감동적으로 볼 수 있었던 영화같습니다^^ 다른분들께도 추천해드립니다.,1
29803,6209860,"스스로를 이해하는 과정, 성장.",1
58151,7447952,최고임,1
68799,10054719,평점 잘못된거 같아서 10점 등록,1
58018,8035444,재이없다재이없다재이없다재이없다재이없다재이없다,1


#### 토큰화 -Okt 사용 (Open Korea Text)

In [42]:
from konlpy.tag import Okt
okt = Okt()

In [50]:
#불용어 설정

stopwords = ['요', '도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [52]:
# Train 데이터에서 불용어 제거

train['document'] = train['document'].astype(str)
train['tokenized'] = train['document'].apply(okt.morphs)
train['tokenized'] = train['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [53]:
# Validation 데이터에서 불용어 제거
val['document'] = val['document'].astype(str)
val['tokenized'] = val['document'].apply(okt.morphs)
val['tokenized'] = val['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [54]:
train.head()

,id,document,label,tokenized
172646,10017356,이런거 안남기는데 솔직히 억지가 너무 심했다. 여자친구랑 다투고 할거없어서 봤는데도...,0,"[이런거, 안, 남기는데, 솔직히, 억지, 너무, 심했다, ., 여자친구, 랑, 다..."
143089,7874090,내용이 너무 지루하다....... 재미없다 굿닥터로 채널 돌림,0,"[내용, 너무, 지루하다, ......., 재미없다, 굿닥터, 로, 채널, 돌림]"
170420,5289473,뭣같은 다큐멘터리도 구천원씩 주고 극장에서 보게 해야겠냐?? 걍 티비에서 방영 하라,0,"[뭣같, 다큐멘터리, 구천원, 씩, 주고, 극장, 에서, 보게, 해야겠냐, ??, ..."
188793,2259841,이게 영화냐?,0,"[영화, 냐, ?]"
56748,6698652,인간의 본성에 대한 철처한 해부.,1,"[인간, 본성, 대한, 철처, 해부, .]"


#### 데이터에 정수 인코딩

In [ ]:
X_train = train['tokenized']
y_train = train['label']
X_val= val['tokenized']
y_val = val['label']

In [ ]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
# 일단 토큰화를 진행함
# total_cnt는 전체 단어 집합의 수
# rare_cnt는 휘기 단어의 수

threshold = 2
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if(value < threshold):
    # value가 threshold를 넘지 않으면 실행
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 학습에 필요한 vocab의 크기
# 희귀 단어는 필요없다고 판단하여 크기 빼줌
vocab_size = total_cnt - rare_cnt
print('단어 집합의 크기 :',vocab_size)

#### GRU 모델을 이용한 감성 분석

In [39]:
#필요한 라이브러리 다운로드
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [40]:
#기본적인 GRU 모델
model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(GRU(128))
model.add(Dense(1, activation='relu'))

NameError: name 'vocab_size' is not defined

In [41]:
model.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
2134/2134 [==============================] - 97s 42ms/step - loss: 0.4915 - acc: 0.8743 - val_loss: 0.4012 - val_acc: 0.8977

Epoch 00001: val_acc improved from -inf to 0.89772, saving model to best_model.h5
Epoch 2/15
2134/2134 [==============================] - 89s 42ms/step - loss: 0.4183 - acc: 0.9072 - val_loss: 0.4295 - val_acc: 0.9093

Epoch 00002: val_acc improved from 0.89772 to 0.90931, saving model to best_model.h5
Epoch 3/15
2134/2134 [==============================] - 89s 42ms/step - loss: 0.3726 - acc: 0.9183 - val_loss: 0.4749 - val_acc: 0.9104

Epoch 00003: val_acc improved from 0.90931 to 0.91037, saving model to best_model.h5
Epoch 4/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.3433 - acc: 0.9252 - val_loss: 0.4438 - val_acc: 0.9099

Epoch 00004: val_acc did not improve from 0.91037
Epoch 5/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.3028 - acc: 0.9323 - val_loss: 0.3885 - val_acc: 0.9128

Epoch 00005: 

#### LSTM 모델을 이용한 감성 분석

In [ ]:
from tensorflow.keras.layers import LSTM, Bidirectional

model_lstm = Sequential()
model_lstm.add(Embedding(vocab_size, 100))
model_lstm.add(LSTM(100)) #Bidirectional drop
model_lstm.add(Dense(1, activation='relu')) #activation = sigmoid -> relu

In [ ]:
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         4361200   
_________________________________________________________________
lstm (LSTM)                  (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 4,441,701
Trainable params: 4,441,701
Non-trainable params: 0
_________________________________________________________________


In [ ]:
es_lstm = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc_lstm = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [ ]:
model_lstm.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)

Epoch 1/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.2797 - acc: 0.9742 - val_loss: 1.0265 - val_acc: 0.9019

Epoch 00001: val_acc did not improve from 0.91481
Epoch 2/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.2739 - acc: 0.9752 - val_loss: 1.0494 - val_acc: 0.8963

Epoch 00002: val_acc did not improve from 0.91481
Epoch 3/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.2722 - acc: 0.9769 - val_loss: 1.2365 - val_acc: 0.8906

Epoch 00003: val_acc did not improve from 0.91481
Epoch 4/15
2134/2134 [==============================] - 89s 42ms/step - loss: 0.2691 - acc: 0.9779 - val_loss: 1.1878 - val_acc: 0.8982

Epoch 00004: val_acc did not improve from 0.91481
Epoch 5/15
2134/2134 [==============================] - 88s 41ms/step - loss: 0.2717 - acc: 0.9790 - val_loss: 1.2105 - val_acc: 0.8943

Epoch 00005: val_acc did not improve from 0.91481
Epoch 00005: early stopping


In [ ]:
y_pred = model_lstm.predict(X_val)

#### model predict

In [ ]:
print(val['reviews'])

126682                                     조립하기힘들었어요 나사가짧아서
64852         ㅜㅜ 제가 인데 ㅜㅜ 진짜 너무너무커요 빨으니까 더 커졌어요ㅜ 잠옷으로 입으려구요
139941                                        만족하는데 배송이 개느림
85091                         허리받쳐주는건 좋은데 방석이 미끌어져서 뷸편하네용 ㅠ
61522     밥을 해도 압력이 안된 밥을 먹었는데 새 패킹으로 갈아끼우니 짱짱하네여내일부터는 밥...
                                ...                        
142347                   라면 냄비로 구입 했었는데 제가 생각 했던것보다 많이 작았네요
145373    좋았습니다 주문을 미루고 미루다보니 시간이 급해서 당일제작 옵션 선택해서 진행했습니...
146410                      최악이다 기포 안없어져서 다시 뜯었는데 그대로 굳었어 ㅋ
122213                                       재질도 좋고 색감도 예뻐요
130757    끌로에 화장대거울스툴 구매하고 만족스러워서 서랍장 하나 더 구매했습니다 근데 하필 ...
Name: reviews, Length: 40000, dtype: object


> GRU



In [ ]:
def sentiment_predict_gru(sentence):
  sentence = okt.morphs(sentence) # 토큰화
  sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
sentiment_predict_gru('좋았습니다 주문을 미루고 미루다보니 시간이 급해서 당일제작 옵션 선택해서 진행했습니다.')

117.19% 확률로 긍정 리뷰입니다.


In [ ]:
sentiment_predict_gru('재질도 좋고 색감도 예뻐요 만족스러워서 서랍장 하나 더 구매했습니다')

356.09% 확률로 긍정 리뷰입니다.


In [ ]:
sentiment_predict_gru('라면 냄비로 구입 했었는데 제가 생각 했던것보다 많이 작았네요')

91.62% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict_gru('최악이다 기포 안없어져서 다시 뜯었는데 그대로 굳었어 ㅋ')

100.00% 확률로 부정 리뷰입니다.


> BiLSTM

In [ ]:
def sentiment_predict_lstm(sentence):
  sentence = okt.morphs(sentence) # 토큰화
  sentence = [word for word in sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(model_lstm.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.".format((1 - score) * 100))

In [ ]:
sentiment_predict_lstm('좋았습니다 주문을 미루고 미루다보니 시간이 급해서 당일제작 옵션 선택해서 진행했습니다.')

100.00% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict_lstm('재질도 좋고 색감도 예뻐요 만족스러워서 서랍장 하나 더 구매했습니다')

100.00% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict_lstm('라면 냄비로 구입 했었는데 제가 생각 했던것보다 많이 작았네요')

99.62% 확률로 부정 리뷰입니다.


In [ ]:
sentiment_predict_lstm('최악이다 기포 안없어져서 다시 뜯었는데 그대로 굳었어 ㅋ')

98.34% 확률로 부정 리뷰입니다.


#### RNN 적용하기


In [ ]:
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense
from tensorflow.keras.models import Sequential

embedding_dim = 32
hidden_units =32



model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(SimpleRNN(hidden_units))
model.add(Dense(1, activation='sigmoid'))
model.summary()

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train_padded, y_train, epochs=4, batch_size=64, validation_split=0.2)

In [ ]:
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_encoded, maxlen = max_len)
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test_padded, y_test)[1])